In [6]:
# Querying Yahoo Finance Data

In [7]:
import requests
import pandas as pd
import numpy as np

# Data Retrieval

In [46]:
# Captures the Equity page to retrieve Expiration Dates

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

req = requests.get('http://query1.finance.yahoo.com/v7/finance/options/SPY', headers = headers)

In [258]:
# Request Time
req.headers['date']

'Tue, 25 Jan 2022 21:49:58 GMT'

In [153]:
# Decode the response
dates = pd.read_json(req.content.decode())

# Translate Unix Timestamps into Python TimeDate 
dates_unix = dates.optionChain[1][0]['expirationDates']
dates_timedate = pd.to_datetime(dates_unix, origin='unix', unit = 's')

# Convert to Human-readable form
dates_human = []
for i, each in enumerate(dates_timedate):
    dates_human.append(str(each.year) + '-' + str(each.month) + '-' + str(each.day))


In [154]:
# Request Option chains for each corresponding Expiration Date (Human-Readable)

all_exp = {}
for i, exp_date in enumerate(dates_unix):
    all_exp[dates_human[i]] = pd.read_json(requests.get('http://query1.finance.yahoo.com/v7/finance/options/SPY' + '?date=' + str(exp_date), headers = headers).content.decode())

In [155]:
# Assure the dictionary contains all Expiration Dates of interest
all_exp.keys()

dict_keys(['2022-1-26', '2022-1-28', '2022-1-31', '2022-2-2', '2022-2-4', '2022-2-7', '2022-2-9', '2022-2-11', '2022-2-14', '2022-2-16', '2022-2-18', '2022-2-22', '2022-2-23', '2022-2-25', '2022-2-28', '2022-3-4', '2022-3-18', '2022-3-31', '2022-4-14', '2022-5-20', '2022-6-17', '2022-6-30', '2022-9-16', '2022-9-30', '2022-12-16', '2022-12-30', '2023-1-20', '2023-3-17', '2023-6-16', '2023-12-15', '2024-1-19', '2024-12-20'])

In [156]:
# Convert the format from Dictionaties to Pandas Dataframe
# Each Chain can now be accessed by quering 'YY-MM-DD' Expiration date in the Dictionary

for each in all_exp:
    all_exp[each] = pd.DataFrame.from_dict(all_exp[each].optionChain.result[0], orient='index')

In [162]:
all_exp['2023-3-17']

,0
underlyingSymbol,SPY
expirationDates,"[1643155200, 1643328000, 1643587200, 164376000..."
strikes,"[150.0, 160.0, 170.0, 180.0, 190.0, 195.0, 200..."
hasMiniOptions,False
quote,"{'language': 'en-US', 'region': 'US', 'quoteTy..."
options,"[{'expirationDate': 1679011200, 'hasMiniOption..."


# Datapoints of Interest

## Equity

In [202]:
# Information Accessible for a Single Equity
all_exp['2023-3-17'][0][4].keys()

dict_keys(['language', 'region', 'quoteType', 'quoteSourceName', 'triggerable', 'currency', 'trailingAnnualDividendRate', 'trailingPE', 'trailingAnnualDividendYield', 'ytdReturn', 'trailingThreeMonthReturns', 'trailingThreeMonthNavReturns', 'epsTrailingTwelveMonths', 'sharesOutstanding', 'bookValue', 'fiftyDayAverage', 'fiftyDayAverageChange', 'fiftyDayAverageChangePercent', 'twoHundredDayAverage', 'twoHundredDayAverageChange', 'twoHundredDayAverageChangePercent', 'marketCap', 'priceToBook', 'sourceInterval', 'exchangeDataDelayedBy', 'tradeable', 'exchangeTimezoneName', 'exchangeTimezoneShortName', 'gmtOffSetMilliseconds', 'esgPopulated', 'firstTradeDateMilliseconds', 'priceHint', 'postMarketChangePercent', 'postMarketTime', 'postMarketPrice', 'postMarketChange', 'regularMarketChange', 'regularMarketChangePercent', 'regularMarketTime', 'regularMarketPrice', 'regularMarketDayHigh', 'regularMarketDayRange', 'regularMarketDayLow', 'regularMarketVolume', 'regularMarketPreviousClose', 'bid'

In [236]:
# All Expiration Dates available in UNIX Timestamp format
all_exp['2023-3-17'][0]['expirationDates']

[1643155200,
 1643328000,
 1643587200,
 1643760000,
 1643932800,
 1644192000,
 1644364800,
 1644537600,
 1644796800,
 1644969600,
 1645142400,
 1645488000,
 1645574400,
 1645747200,
 1646006400,
 1646352000,
 1647561600,
 1648684800,
 1649894400,
 1653004800,
 1655424000,
 1656547200,
 1663286400,
 1664496000,
 1671148800,
 1672358400,
 1674172800,
 1679011200,
 1686873600,
 1702598400,
 1705622400,
 1734652800]

## Option Chain

In [239]:
# Datapoints available for Each Contract
all_exp['2023-3-17'].loc['options'][0][0]['calls'][0].keys()

dict_keys(['contractSymbol', 'strike', 'currency', 'lastPrice', 'change', 'percentChange', 'volume', 'openInterest', 'bid', 'ask', 'contractSize', 'expiration', 'lastTradeDate', 'impliedVolatility', 'inTheMoney'])

In [243]:
# Current Expiration Date
all_exp['2023-3-17'].loc['options'][0][0]['expirationDate']

# All Puts and Calls in order (Calls & Puts: Low to High Strike)
all_exp['2023-3-17'].loc['options'][0][0]['calls']
all_exp['2023-3-17'].loc['options'][0][0]['puts']

# Strike Prices for a Current Expiration Date
all_exp['2023-3-17'].loc['strikes'][0]

# Number of different contracts by Expirations
len(all_exp['2023-3-17'].loc['options'][0][0]['puts'])

86

In [254]:
# Access to a specific Contract #15
all_exp['2023-3-17'].loc['options'][0][0]['puts'][15]

# Access to a specific Datapoint
all_exp['2023-3-17'].loc['options'][0][0]['puts'][15]['impliedVolatility']

0.40735455932617104

# Dataset

## Construction

## Saving & File Structure